In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

import numpy as np
import pandas as pd

In [11]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2023-02-27 06:54:13--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 142.251.10.139, 142.251.10.100, 142.251.10.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.10.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/osaqagb89dct70ktop2pnjnl73osfto7/1677480825000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=0c71f2a0-87ae-48dc-9716-76deaed98209 [following]
--2023-02-27 06:54:15--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/osaqagb89dct70ktop2pnjnl73osfto7/1677480825000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=0c71f2a0-87ae-48dc-9716-76deaed98209
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 172.217.194.132, 2404:6800:4003:c04::84
Connecting to doc-0

In [13]:
dataset = pd.read_csv('/tmp/sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist() 

In [35]:
import tensorflow_datasets as tfds

In [36]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(sentences, 1000, max_subword_length=5)

In [37]:
num = 5
print(sentences[num])
encoded = tokenizer.encode(sentences[num])
print(encoded)
# Separately print out each subword, decoded
for i in encoded:
  print(tokenizer.decode([i]))

I have to jiggle the plug to get it to line up right to get decent volume.
[4, 31, 6, 849, 162, 450, 12, 1, 600, 438, 775, 6, 175, 14, 6, 55, 213, 159, 474, 775, 6, 175, 614, 380, 295, 148, 72, 789]
I 
have 
to 
j
ig
gl
e 
the 
pl
ug
 
to 
get 
it 
to 
li
ne 
up 
right
 
to 
get 
dec
ent 
vo
lu
me
.


In [38]:
training_size = int(len(sentences)*0.8)
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [39]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type = 'post'
oov_token = '<OOV>'

In [40]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

In [41]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [42]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [20]:
print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


In [49]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 16)           16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_4 (Dense)             (None, 6)                 102       
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 16,109
Trainable params: 16,109
Non-trainable params: 0
_________________________________________________________________


In [50]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs = num_epochs, validation_data = (testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 5s 82ms/step - loss: 0.6927 - accuracy: 0.5129 - val_loss: 0.6998 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 2s 33ms/step - loss: 0.6912 - accuracy: 0.5223 - val_loss: 0.7008 - val_accuracy: 0.4110
Epoch 3/10
50/50 [==============================] - 1s 15ms/step - loss: 0.6900 - accuracy: 0.5223 - val_loss: 0.6990 - val_accuracy: 0.4110
Epoch 4/10
50/50 [==============================] - 1s 11ms/step - loss: 0.6879 - accuracy: 0.5235 - val_loss: 0.6960 - val_accuracy: 0.4135
Epoch 5/10
50/50 [==============================] - 1s 14ms/step - loss: 0.6838 - accuracy: 0.5374 - val_loss: 0.6942 - val_accuracy: 0.4261
Epoch 6/10
50/50 [==============================] - 1s 13ms/step - loss: 0.6772 - accuracy: 0.5788 - val_loss: 0.6919 - val_accuracy: 0.4486
Epoch 7/10
50/50 [==============================] - 1s 22ms/step - loss: 0.6686 - accuracy: 0.6478 - val_loss: 0.6812 - val_accuracy: 0.5363
Epoch 8/10
50

In [45]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(1000, 16)


In [28]:
import io

In [51]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [52]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

In [32]:
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding = padding_type, maxlen = max_length)

In [33]:
classes = model.predict(fakes_padded)

1/1 [==============================] - 0s 75ms/step


In [34]:
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

I love this phone
[0.9569447]


I hate spaghetti
[0.09018662]


Everything was cold
[0.573902]


Everything was hot exactly as I wanted
[0.8493716]


Everything was green
[0.61078334]


the host seated us immediately
[0.7563023]


they gave us free chocolate cake
[0.7769362]


not sure about the wilted flowers on the table
[0.0353915]


only works when I stand on tippy toes
[0.7142498]


does not work when I stand on my head
[0.00342589]


